In [2]:
# Load Packages
from datetime import date, datetime, timedelta
import calendar
import pandas as pd
import numpy as np
import itertools
import copy
from numpy.linalg import multi_dot
from scipy.stats import norm
from scipy.stats import bernoulli
import matplotlib.pyplot as plt
import math

#Import Date opertative user defined functions
from ipynb.fs.full.user_defined_vik_functions import get_all_monthly_option_expiries, \
                                                     find_last_thurs_date_of_month, \
                                                     prev_workday_if_holiday, find_wkly_expries,\
                                                     date_of_prev_thurs, generate_daily_dates_each_month, \
                                                     next_workday, next_weekday, \
                                                     create_df_wk_days, find_week, date_of_next_thurs

# Import dataframe naming functions 
from ipynb.fs.full.user_defined_vik_functions import get_mthly_df_name_from_expiry

#Import data loading functions
from ipynb.fs.full.user_defined_vik_functions import load_all_mthly_data


In [17]:
#Compute Daily Static  Hedging Errors

###################################
# Common Variables initialisation #
###################################

# Stock Index of Interest
# stock_ident = "BANKNIFTY"
stock_ident = "NIFTY"

#Static hedging performed at different moneyness regions 
#i.e. moneyness is used to select the option with nearest moneyness match
# ATM - At the Money, ITM - In the money, OTM - Out of the Money
prod_moneyness = "ATM"

#Product type to hedge: either "CE" or "PE"
prod_type = "CE"
# prod_type = "PE"

# code_id_dict_smile_sim = {"Dynamic": "3", "ConstVol": "11", "ConstVolSmile": "12", "ConstVolSurf": "13", "FwdSmile": "14"}
code_id_dict_atm_sim = {"Dynamic": "1", "ConstVol": "1A", "ConstVolSmile": "2A", "ConstVolSurf": "3A", "FwdSmile": "4A"}
code_id_dict_carrWu = {"Dynamic": "1", "ConstVol": "1A", "CarrWu": "0"}
code_id_dict = {"atmSim": code_id_dict_atm_sim, "carrWu": code_id_dict_carrWu}
# code_id_dict = {"smileSim": code_id_dict_smile_sim, "atmSim": code_id_dict_atm_sim, "carrWu": code_id_dict_carrWu}


if prod_type == "CE":
#     code_ids_dict_list = ["smileSim", "atmSim", "carrWu"]
    code_ids_dict_list = ["atmSim", "carrWu"]
else:
#     code_ids_dict_list = ["smileSim", "atmSim"] 
    code_ids_dict_list = ["atmSim"] 
    
code_id_output_map = {"smileSim": "1", "atmSim": "1A", "carrWu": "0"}

#Path to refer data
source_path = "/home/jupyter-partha/Vikranth - Chapter 2/"
input_sub_path = "Input Data/mkt_data_covid_region/"
output_sub_path = "Output Data/"
input_data_path = source_path + input_sub_path
output_data_path = source_path + output_sub_path


# Periods of interest will be a dictionary
#Key is the year, value is a list of months 1-12, 1- Jan, 2 - Feb,...12 - Dec
# For E.g., periods_of_interest = {2020: [3], 2019: [11, 12]}
periods_of_interest = {2019: [8, 9, 10, 11, 12], 2020: [1, 2, 3, 4, 5, 6, 7]}
# periods_of_interest = {2019: [8, 9, 10, 11, 12], 2020: [1, 2, 3, 4, 5, 6]}

#List of holidays
holidays_list = [date(2019, 3, 4), date(2019, 3, 21),\
                 date(2019, 4, 17), date(2019, 4, 19), date(2019, 4, 29),\
                 date(2019, 5, 1),\
                 date(2019, 6, 5),\
                 date(2019, 8, 12), date(2019, 8, 15),\
                 date(2019, 9, 2), date(2019, 9, 10), \
                 date(2019, 10, 2), date(2019, 10, 8), date(2019, 10, 21), date(2019, 10, 28), \
                 date(2019, 11, 12), \
                 date(2019, 12, 25), \
                 date(2020, 2, 21), \
                 date(2020, 3, 10), \
                 date(2020, 4, 2), date(2020, 4, 6), date(2020, 4, 10), date(2020,4, 14), \
                 date(2020, 5, 1), date(2020, 5, 25), \
                 date(2020, 10, 2), date(2020, 11, 16), date(2020, 11, 30), date(2020, 12, 25)]


In [18]:

for code_id_str in code_ids_dict_list:

    #Find the monthly strike of option from mkt data to find the option to be hedged
    #Every month, an option is hedged
    mthly_expiries_list = get_all_monthly_option_expiries(periods_of_interest, holidays_list)
    dict_wkly_expiries_each_month = find_wkly_expries(mthly_expiries_list, holidays_list)

    #Input all model hedge outputs
    df_dynamic_init = pd.read_csv(output_data_path + "D" + code_id_dict[code_id_str]["Dynamic"] + "_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_dynamic_hedge.csv")
    df_dynamic_init['Dynamic Hedge Abs Error'] = df_dynamic_init.loc[:, 'Dynamic Hedge Error'].abs()
    dynamic_PnL_dfs = []

    # Hedge Cost
    #0.05% of premium - Options - Only on the day of entering, hedge value * 0.0005
    # 0.00345% of stock price - (delta - delta_lag) * Underlying price * 0.0000345

    for each_month in mthly_expiries_list:
        df_dyn = df_dynamic_init[df_dynamic_init['Month_Expiry'] == each_month.strftime("%d-%b-%Y")]
        df_dyn['Close_lag'] = [list(df_dyn['Close'])[0]] + list(df_dyn['Close'])[:-1]
        df_dyn['Date_lag'] = [list(df_dyn['Date'])[0]] + list(df_dyn['Date'])[:-1]
        df_dyn['dynamic_pfl_lag'] = [list(df_dyn['Dynamic Hedge Pfl Value'])[0]] + list(df_dyn['Dynamic Hedge Pfl Value'])[:-1]
        df_dyn['Pfl PnL - Realised'] = df_dyn['Close'] - df_dyn['Close_lag']
        df_dyn['Underlying Value - Lag'] = [list(df_dyn['Underlying Value'])[0]] + list(df_dyn['Underlying Value'])[:-1]
        # The following statement is wrong for dyamic hedge - this is wrong
        #     df_dyn['Pfl PnL - Dynamic Hedge'] = df_dyn['Dynamic Hedge Pfl Value'] - df_dyn['dynamic_pfl_lag']
        df_dyn['Pfl PnL - Dynamic'] = df_dyn['Dynamic Hedge Pfl Value'] - df_dyn['Close_lag']
        df_dyn = df_dyn[df_dyn['Date'] != df_dyn['Date_lag']]
        df_dyn = df_dyn.rename(columns={'Dynamic Hedge Pfl Value': 'Hedge Pfl Value - Dynamic'})
        df_dyn = df_dyn[['Date', 'Month_Expiry', 'Underlying Value', 'Underlying Value - Lag', \
                         'Date_lag',  'Strike Price', 'r', 'expiry_dt', \
                         'Close', 'Close_lag', 'Pfl PnL - Realised', \
                         'iv', 'delta', 'delta_lag', \
                         'Hedge Pfl Value - Dynamic', 'dynamic_pfl_lag', \
                         'Pfl PnL - Dynamic']]
        dynamic_PnL_dfs.append(df_dyn)

    df_dynamic_PnL = pd.concat(dynamic_PnL_dfs, axis=0)

    # Static Hedge PnLs
    df_const_vol = pd.read_csv(output_data_path + "C" + code_id_dict[code_id_str]["ConstVol"] +"_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + ".csv")
    
    if (code_id_str == "carrWu"):
        df_carrWu = pd.read_csv(output_data_path + "C" + code_id_dict[code_id_str]["CarrWu"] + "_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + ".csv")
    else:
        df_const_volSmile = pd.read_csv(output_data_path + "C" + code_id_dict[code_id_str]["ConstVolSmile"] + "_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + ".csv")
        df_const_volSurf = pd.read_csv(output_data_path + "C" + code_id_dict[code_id_str]["ConstVolSurf"] + "_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + ".csv")
        df_fwdSmile = pd.read_csv(output_data_path + "C" + code_id_dict[code_id_str]["FwdSmile"] + "_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + ".csv")

    df_const_vol_PnL_dfs=[]
    
    if (code_id_str == "carrWu"):
        df_carrWu_PnL_dfs=[]
    else:
        df_const_volSmile_PnL_dfs=[]
        df_const_volSurf_PnL_dfs=[]
        df_fwdSmile_PnL_dfs=[]


    for each_month in mthly_expiries_list:
        df_const_vol_PnL_wkly_dfs = []
        
        if (code_id_str == "carrWu"):
            df_CarrWu_PnL_wkly_dfs = []
        else:
            df_const_volSmile_PnL_wkly_dfs = []
            df_const_volSurf_PnL_wkly_dfs = []
            df_fwdSmile_PnL_wkly_dfs = []            

        df_const_vol_mth = df_const_vol[df_const_vol['Month_Expiry'] == each_month.strftime("%d-%b-%Y")]
        
        if (code_id_str == "carrWu"):
            df_carrWu_mth = df_carrWu[df_carrWu['Month_Expiry'] == each_month.strftime("%d-%b-%Y")]
        else:
            df_const_volSmile_mth = df_const_volSmile[df_const_volSmile['Month_Expiry'] == each_month.strftime("%d-%b-%Y")]
            df_const_volSurf_mth = df_const_volSurf[df_const_volSurf['Month_Expiry'] == each_month.strftime("%d-%b-%Y")]
            df_fwdSmile_mth = df_fwdSmile[df_fwdSmile['Month_Expiry'] == each_month.strftime("%d-%b-%Y")] 
            
        static_trans_cost  = 0.0005
        
        for each_week in dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")]:
            
            #Const Vol
            df_const_vol_wk = df_const_vol_mth[df_const_vol_mth['Week_Start'] == each_week.strftime("%d-%b-%Y")]
            df_const_vol_wk['Date_lag'] = [list(df_const_vol_wk['Date'])[0]] + list(df_const_vol_wk['Date'])[:-1]
            df_const_vol_wk['Hedge_Pfl_value_Lag'] = [list(df_const_vol_wk['Hedge_Pfl_value'])[0]] + list(df_const_vol_wk['Hedge_Pfl_value'])[:-1]

            df_const_vol_wk['Hedge_Pfl_value_Lag_with_trans_cost'] = [list(df_const_vol_wk['Hedge_Pfl_value_Lag'])[0] * 1.0005 , list(df_const_vol_wk['Hedge_Pfl_value_Lag'])[0] * 1.0005] + list(df_const_vol_wk['Hedge_Pfl_value_Lag'])[2:]
            df_const_vol_wk['Pfl PnL - ConstVol'] = np.array(df_const_vol_wk['Hedge_Pfl_value']) - df_const_vol_wk['Hedge_Pfl_value_Lag_with_trans_cost']

            df_const_vol_wk = df_const_vol_wk.rename(columns={"Hedge_Pfl_value": "Hedge Pfl Value - ConstVol", "Static Hedge Error": "Static Hedge Error - ConstVol"})
            df_const_vol_wk = df_const_vol_wk[df_const_vol_wk['Date'] != df_const_vol_wk['Date_lag']]
    #         df_const_vol_wk = df_const_vol_wk[["Date", "Month_Expiry", "Week_Start", "Week_Expiry", "ConstVol - Hedge Pfl Value", "Static Hedge Error","Pfl PnL - ConstVol Static Hedge" ]]
            df_const_vol_wk = df_const_vol_wk[["Date", "Month_Expiry", "Week_Start", "Week_Expiry", \
                                               "Hedge Pfl Value - ConstVol", "Static Hedge Error - ConstVol", \
                                               "Pfl PnL - ConstVol" ]]

            df_const_vol_PnL_wkly_dfs.append(df_const_vol_wk)

            if (code_id_str == "carrWu"):
                #Carr Wu
                df_carrWu_wk = df_carrWu_mth[df_carrWu_mth['Week_Start'] == each_week.strftime("%d-%b-%Y")]
                df_carrWu_wk['Date_lag'] = [list(df_carrWu_wk['Date'])[0]] + list(df_carrWu_wk['Date'])[:-1]
                df_carrWu_wk['Hedge_Pfl_value_Lag'] = [list(df_carrWu_wk['Hedge_Pfl_value'])[0]] + list(df_carrWu_wk['Hedge_Pfl_value'])[:-1]
                
                df_carrWu_wk['Hedge_Pfl_value_Lag_with_trans_cost'] = [list(df_carrWu_wk['Hedge_Pfl_value_Lag'])[0] * 1.0005 , list(df_carrWu_wk['Hedge_Pfl_value_Lag'])[0] * 1.0005] + list(df_carrWu_wk['Hedge_Pfl_value_Lag'])[2:]
                df_carrWu_wk['Pfl PnL - CarrWu'] = np.array(df_carrWu_wk['Hedge_Pfl_value']) - df_carrWu_wk['Hedge_Pfl_value_Lag_with_trans_cost']
#                 df_carrWu_wk['Pfl PnL - CarrWu'] = df_carrWu_wk['Hedge_Pfl_value'] - df_carrWu_wk['Hedge_Pfl_value_Lag']
                
                df_carrWu_wk = df_carrWu_wk.rename(columns={"Hedge_Pfl_value": "Hedge Pfl Value - CarrWu", "Static Hedge Error": "Static Hedge Error - CarrWu"})
                df_carrWu_wk = df_carrWu_wk[df_carrWu_wk['Date'] != df_carrWu_wk['Date_lag']]
        #         df_carrWu_wk = df_carrWu_wk[["Date", "Month_Expiry", "Week_Start", "Week_Expiry", "CarrWu - Hedge Pfl Value", "Static Hedge Error (Carr Wu)", "Pfl PnL - CarrWu Static Hedge" ]]
                df_carrWu_wk = df_carrWu_wk[["Date", "Month_Expiry", "Week_Start", "Week_Expiry", \
                                             "Hedge Pfl Value - CarrWu", "Static Hedge Error - CarrWu", \
                                             "Pfl PnL - CarrWu"]]
                df_CarrWu_PnL_wkly_dfs.append(df_carrWu_wk)                
                
            
            else:
                #Const Vol Smile
                df_const_volsmile_wk = df_const_volSmile_mth[df_const_volSmile_mth['Week_Start'] == each_week.strftime("%d-%b-%Y")]
                df_const_volsmile_wk['Date_lag'] = [list(df_const_volsmile_wk['Date'])[0]] + list(df_const_volsmile_wk['Date'])[:-1]
                df_const_volsmile_wk['Hedge_Pfl_value_Lag'] = [list(df_const_volsmile_wk['Hedge_Pfl_value'])[0]] + list(df_const_volsmile_wk['Hedge_Pfl_value'])[:-1]
                df_const_volsmile_wk['Hedge_Pfl_value_Lag_with_trans_cost'] = [list(df_const_volsmile_wk['Hedge_Pfl_value_Lag'])[0] * 1.0005 , list(df_const_volsmile_wk['Hedge_Pfl_value_Lag'])[0] * 1.0005] + list(df_const_volsmile_wk['Hedge_Pfl_value_Lag'])[2:]
                df_const_volsmile_wk['Pfl PnL - ConstVolSmile'] = np.array(df_const_volsmile_wk['Hedge_Pfl_value']) - df_const_volsmile_wk['Hedge_Pfl_value_Lag_with_trans_cost']

#                 df_const_volsmile_wk['Pfl PnL - ConstVolSmile'] = df_const_volsmile_wk['Hedge_Pfl_value'] - df_const_volsmile_wk['Hedge_Pfl_value_Lag']
                df_const_volsmile_wk = df_const_volsmile_wk.rename(columns={'Hedge_Pfl_value': 'Hedge Pfl Value - ConstVolSmile', "Static Hedge Error": "Static Hedge Error - ConstVolSmile"})
                df_const_volsmile_wk = df_const_volsmile_wk[df_const_volsmile_wk['Date'] != df_const_volsmile_wk['Date_lag']]
        #         df_const_volsmile_wk = df_const_volsmile_wk[["Date", "Month_Expiry", "Week_Start", "Week_Expiry", "ConstVolSmile - Hedge Pfl Value", "Static Hedge Error","Pfl PnL - ConstVolSmile Static Hedge"]]
                df_const_volsmile_wk = df_const_volsmile_wk[["Date", "Month_Expiry", "Week_Start", "Week_Expiry", \
                                                             "Hedge Pfl Value - ConstVolSmile", "Static Hedge Error - ConstVolSmile", \
                                                             "Pfl PnL - ConstVolSmile"]]
                df_const_volSmile_PnL_wkly_dfs.append(df_const_volsmile_wk)

                #Const Vol Surface
                df_const_volsurf_wk = df_const_volSurf_mth[df_const_volSurf_mth['Week_Start'] == each_week.strftime("%d-%b-%Y")]
                df_const_volsurf_wk['Date_lag'] = [list(df_const_volsurf_wk['Date'])[0]] + list(df_const_volsurf_wk['Date'])[:-1]
                df_const_volsurf_wk['Hedge_Pfl_value_Lag'] = [list(df_const_volsurf_wk['Hedge_Pfl_value'])[0]] + list(df_const_volsurf_wk['Hedge_Pfl_value'])[:-1]

                df_const_volsurf_wk['Hedge_Pfl_value_Lag_with_trans_cost'] = [list(df_const_volsurf_wk['Hedge_Pfl_value_Lag'])[0] * 1.0005 , list(df_const_volsurf_wk['Hedge_Pfl_value_Lag'])[0] * 1.0005] + list(df_const_volsurf_wk['Hedge_Pfl_value_Lag'])[2:]
                df_const_volsurf_wk['Pfl PnL - ConstVolSurf'] = np.array(df_const_volsurf_wk['Hedge_Pfl_value']) - df_const_volsurf_wk['Hedge_Pfl_value_Lag_with_trans_cost']

#                 df_const_volsurf_wk['Pfl PnL - ConstVolSurf'] = df_const_volsurf_wk['Hedge_Pfl_value'] - df_const_volsurf_wk['Hedge_Pfl_value_Lag']
                df_const_volsurf_wk = df_const_volsurf_wk.rename(columns={'Hedge_Pfl_value': 'Hedge Pfl Value - ConstVolSurf', "Static Hedge Error": "Static Hedge Error - ConstVolSurf"})
                df_const_volsurf_wk = df_const_volsurf_wk[df_const_volsurf_wk['Date'] != df_const_volsurf_wk['Date_lag']]
        #         df_const_volsurf_wk = df_const_volsurf_wk[["Date", "Month_Expiry", "Week_Start", "Week_Expiry", "ConstVolSurf - Hedge Pfl Value", "Static Hedge Error","Pfl PnL - ConstVolSurf Static Hedge"]]
                df_const_volsurf_wk = df_const_volsurf_wk[["Date", "Month_Expiry", "Week_Start", "Week_Expiry", \
                                                           "Hedge Pfl Value - ConstVolSurf", "Static Hedge Error - ConstVolSurf", \
                                                           "Pfl PnL - ConstVolSurf"]]
                df_const_volSurf_PnL_wkly_dfs.append(df_const_volsurf_wk)


                #Fwd Smile
                df_fwdSmile_wk = df_fwdSmile_mth[df_fwdSmile_mth['Week_Start'] == each_week.strftime("%d-%b-%Y")]
                df_fwdSmile_wk['Date_lag'] = [list(df_fwdSmile_wk['Date'])[0]] + list(df_fwdSmile_wk['Date'])[:-1]
                df_fwdSmile_wk['Hedge_Pfl_value_Lag'] = [list(df_fwdSmile_wk['Hedge_Pfl_value'])[0]] + list(df_fwdSmile_wk['Hedge_Pfl_value'])[:-1]

                df_fwdSmile_wk['Hedge_Pfl_value_Lag_with_trans_cost'] = [list(df_fwdSmile_wk['Hedge_Pfl_value_Lag'])[0] * 1.0005 , list(df_fwdSmile_wk['Hedge_Pfl_value_Lag'])[0] * 1.0005] + list(df_fwdSmile_wk['Hedge_Pfl_value_Lag'])[2:]
                df_fwdSmile_wk['Pfl PnL - FwdSmile'] = np.array(df_fwdSmile_wk['Hedge_Pfl_value']) - df_fwdSmile_wk['Hedge_Pfl_value_Lag_with_trans_cost']

#                 df_fwdSmile_wk['Pfl PnL - FwdSmile Static Hedge'] = df_fwdSmile_wk['Hedge_Pfl_value'] - df_fwdSmile_wk['Hedge_Pfl_value_Lag']
                df_fwdSmile_wk = df_fwdSmile_wk.rename(columns={'Hedge_Pfl_value': 'Hedge Pfl Value - FwdSmile', "Static Hedge Error": "Static Hedge Error - FwdSmile"})
                df_fwdSmile_wk = df_fwdSmile_wk[df_fwdSmile_wk['Date'] != df_fwdSmile_wk['Date_lag']]
        #         df_fwdSmile_wk = df_fwdSmile_wk[["Date", "Month_Expiry", "Week_Start", "Week_Expiry", "FwdSmile - Hedge Pfl Value", "Static Hedge Error", "Pfl PnL - FwdSmile Static Hedge"]]
                df_fwdSmile_wk = df_fwdSmile_wk[["Date", "Month_Expiry", "Week_Start", "Week_Expiry", \
                                                 "Hedge Pfl Value - FwdSmile", "Static Hedge Error - FwdSmile", \
                                                 "Pfl PnL - FwdSmile"]]
                df_fwdSmile_PnL_wkly_dfs.append(df_fwdSmile_wk)


        #Combining weekly dfs to month level
        df_const_vol_mth_PnL = pd.concat(df_const_vol_PnL_wkly_dfs, axis=0)
        df_const_vol_PnL_dfs.append(df_const_vol_mth_PnL)

        if (code_id_str != "carrWu"):
            df_const_volsmile_mth_PnL = pd.concat(df_const_volSmile_PnL_wkly_dfs, axis=0)
            df_const_volSmile_PnL_dfs.append(df_const_volsmile_mth_PnL)

            df_const_volsurf_mth_PnL = pd.concat(df_const_volSurf_PnL_wkly_dfs, axis=0)
            df_const_volSurf_PnL_dfs.append(df_const_volsurf_mth_PnL)

            df_fwdSmile_mth_PnL = pd.concat(df_fwdSmile_PnL_wkly_dfs, axis=0)
            df_fwdSmile_PnL_dfs.append(df_fwdSmile_mth_PnL)
        else:   
            df_carrWu_mth_PnL = pd.concat(df_CarrWu_PnL_wkly_dfs, axis=0)
            df_carrWu_PnL_dfs.append(df_carrWu_mth_PnL)

    # ConstVol PnLs
    df_constVol_Pfl_PnL = pd.concat(df_const_vol_PnL_dfs, axis=0)
    
    if (code_id_str == "carrWu"):
        df_carrWu_Pfl_PnL = pd.concat(df_carrWu_PnL_dfs, axis=0)        
    else:
        #VolSmile PnLs
        df_constVolSmile_Pfl_PnL = pd.concat(df_const_volSmile_PnL_dfs, axis=0)
        #VolSurface PnLs
        df_constVolSurf_Pfl_PnL = pd.concat(df_const_volSurf_PnL_dfs, axis=0)
        #Forward Smile
        df_fwdSmile_Pfl_PnL = pd.concat(df_fwdSmile_PnL_dfs, axis=0)


    # Combine PnLs
    df_hedge_PnLs = df_dynamic_PnL.merge(df_constVol_Pfl_PnL, how="left", left_on=["Month_Expiry", "Date"], right_on=["Month_Expiry", "Date"], \
                                         suffixes=('', ''))
    
    if (code_id_str == "carrWu"):
        df_hedge_PnLs = df_hedge_PnLs.merge(df_carrWu_Pfl_PnL, how="left", left_on=["Month_Expiry", "Date"], right_on=["Month_Expiry", "Date"], \
                                             suffixes=('', '_carr'))
        df_hedge_PnLs = df_hedge_PnLs.drop(["Week_Start_carr", "Week_Expiry_carr"], axis=1)        
        
    else:
    
        df_hedge_PnLs = df_hedge_PnLs.merge(df_constVolSmile_Pfl_PnL, how="left", left_on=["Month_Expiry", "Date"], right_on=["Month_Expiry", "Date"], \
                                             suffixes=('', '_smile'))
        df_hedge_PnLs = df_hedge_PnLs.drop(["Week_Start_smile","Week_Expiry_smile"], axis=1)


        df_hedge_PnLs = df_hedge_PnLs.merge(df_constVolSurf_Pfl_PnL, how="left", left_on=["Month_Expiry", "Date"], right_on=["Month_Expiry", "Date"], \
                                             suffixes=('', '_surf'))
        df_hedge_PnLs = df_hedge_PnLs.drop(["Week_Start_surf","Week_Expiry_surf"], axis=1)


        df_hedge_PnLs = df_hedge_PnLs.merge(df_fwdSmile_Pfl_PnL, how="left", left_on=["Month_Expiry", "Date"], right_on=["Month_Expiry", "Date"], \
                                             suffixes=('', '_fwd'))
        df_hedge_PnLs = df_hedge_PnLs.drop(["Week_Start_fwd","Week_Expiry_fwd"], axis=1)


    df_hedge_PnLs.to_csv(output_data_path + "E" + code_id_output_map[code_id_str] + "_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_hedge_results" + ".csv", index = False)



/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [19]:
# Hedge Cost
#0.05% of premium - Options
# 0.00345% of stock price

# Cost - fraction of premium and equity
# Option transaction cost is divided by 5 to equally divide the option buy cost across 5 days of the week
# option_trans_cost = float(0.0005/5) 
# equity_trans_cost = 0.0000345

# df_hedge['Dynamic Hedge Trans Cost'] = (df_hedge['delta'] - df_hedge['delta_lag']) * df_hedge['Underlying Value'] * equity_trans_cost
# df_hedge['Static Hedge Trans Cost'] = df_hedge['const_vol_hedge_value'] * option_trans_cost


In [10]:
# df_hedge.to_csv(output_data_path + "E1_" + stock_ident + "_hedge_results" + ".csv", index = False)


In [1]:
# print(mthly_expiries_list)

In [2]:
# a = [1 , 2, 3]
# print(a[:-1])

In [3]:
# Date,Month_Expiry,Underlying Value,Underlying Value - Lag,Date_lag,Close,Close_lag,Realised Pfl PnL,iv,delta,delta_lag,Dynamic Hedge Pfl Value,dynamic_pfl_lag,Pfl PnL - Dynamic Hedge,
# Week_Start,Week_Expiry,ConstVol - Hedge Pfl Value,Static Hedge Error (BS Const Vol),Pfl PnL - ConstVol Static Hedge,
# ConstVolSmile - Hedge Pfl Value,Static Hedge Error (BS Const Vol Smile),Pfl PnL - ConstVolSmile Static Hedge




In [ ]:
# l = [0, 1,2,3,4,5]
# print(l[2:])


In [3]:
# code_id_dict = {"Dynamic": "3", "ConstVol": "11", "VolSmile": "12", "VolSurf": "13", "FwdSmile": "14"}
# print(code_id_dict["Dynamic"])